# Carsy

Aplikacja wykorzystuje `NavigationBar`. Po raz kolejny wykorzystamy *dummy data* dostarczone przez obiekt `dataProvider`. Wykorzystamy `verticalScroll` oraz `LazyRow` w kilku różnych konfiguracjach. Będzie to uboga wersja [Fuelio](https://play.google.com/store/apps/details?id=com.kajda.fuelio&hl=pl&gl=US). Applikacja wykorzystuje grupowanie elementów osiągnię przez wykorzystanie trzech `Composable` w pojedynczej `Column`, które są ładowane w zależności od typu zawartości.

<table><tr><td><img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExY2MzYmNiN2NmOTE4NDVmZGU4ZTQzNjQ3ZjVkZDlhNzE3MTc0Y2FmOCZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/UT1CIBlbL1FftHf9Hk/giphy.gif" width="200" /></td><td><img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExYWRlZGExYmI4OTZlNjY4MzZiM2NkZmE4ZWMzN2VlMGVjZjk4YWRhOSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/lx96i0NeTGeYV35NFl/giphy.gif" width="200" /></td><td><img src="https://media3.giphy.com/media/v1.Y2lkPTc5MGI3NjExZDFmZTkxZjIxNzQ0NTJmNDFlM2NhN2NiNTlmYzU2N2ZkZTVhODM3MyZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/WyCanQMjlfmYguKc3l/giphy.gif" width="200" /></td></tr></table>

Rozpocznijmy od konfiguracji skryptów `Gradle` - dodajmy nawigację oraz dodatkowe ikony do bloku `dependencies`

In [ ]:
    implementation 'androidx.navigation:navigation-compose:2.5.3'
    implementation "androidx.compose.material:material-icons-extended:1.4.3"

## Dane

Rozpocznijmy od przygotowania *dummy data* dla aplikacji.

In [ ]:
enum class CostType(val costType: String, val icon: ImageVector) {
    REFUELING("Tankowanie", Icons.Default.LocalGasStation),
    SERVICE("Serwis", Icons.Default.CarRepair),
    PARKING("Parking", Icons.Default.LocalParking),
    INSURANCE("Ubezpieczenie", Icons.Default.AttachMoney),
    TICKET("Mandat", Icons.Default.Error)
}

In [ ]:
data class Cost (
    val type: CostType,
    val date: LocalDate,
    val amount: Int
)

In [ ]:
data class Car (
    val name: String,
    val brand: String,
    val model: String,
    val yearOfProduction: Int,
    val costs: List<Cost>
)

In [ ]:
object DataProvider {

    val cars = listOf(
        Car("Domowy", "Skoda", "Fabia", 2002, generalCosts(100)),
        Car("Służbowy", "BMW", "Coupe", 2015, generalCosts(200)),
        Car("Kolekcjonerski", "Fiat", "125p", 1985, generalCosts(120)),
        Car("Sportowy", "Lamborghini", "Murcielago", 2012, generalCosts(100)),
        Car("Zapasowy", "Skoda", "Superb", 2010, generalCosts(120)),
        Car("SUV", "Skoda", "Kodiaq", 2020, generalCosts(300))
    )

    private fun generalCosts(size: Int) = List(size) {
        Cost(
            CostType.values()[Random.nextInt(CostType.values().size)],
            LocalDate.of(Random.nextInt(2005, 2023), Random.nextInt(1,13), Random.nextInt(1,28)),
            Random.nextInt(5000)
        )
    }

    fun getTimeLineList(costs: List<Cost>) = costs
            .sortedBy { it.date }
            .groupBy { it.date.year }
            .flatMap { (year, costsOfYear) -> costListItemsByMonths(year, costsOfYear) }

    private fun costListItemsByMonths(
        year: Int,
        costsOfYear: List<Cost>
    ) = listOf(
        CostListItem.CostYearItem(year.toString()),
        *costsOfYear.groupBy { it.date.month }
            .flatMap { (month, costsOfMonth) -> costListItemsByDays(month, costsOfMonth) }
            .toTypedArray()
    )

    private fun costListItemsByDays(
        month: Month,
        costsOfMonth: List<Cost>
    ) = listOf(
        CostListItem.CostMonthItem(polishMonthsNames(month)),
        *costsOfMonth.sortedBy { it.date.dayOfMonth }
            .map { CostListItem.CostGeneralItem(it) }
            .toTypedArray()
    )
}

In [ ]:
sealed class CostListItem() {
    class CostGeneralItem(val cost: Cost) : CostListItem()
    class CostMonthItem(val month: String) : CostListItem()
    class CostYearItem(val year: String) : CostListItem()
}

Obiekt `DataProvider` zawiera listę obiektów `Car` oraz kilka pomocniczych funkcji związanych z kosztami i manipulacją danymi.

Każdy obiekt `Car` reprezentuje samochód i zawiera takie informacje jak nazwa, marka, model, rok produkcji oraz lista kosztów związanych z tym samochodem. Funkcja `generalCosts()`,  przyjmuje jako parametr rozmiar i generuje listę losowych kosztów. Każdy koszt tworzony jest losowo. 

Funkcja `getTimeLineList()` przyjmuje listę kosztów jako argument i zwraca listę kosztów w formie osi czasu. Najpierw sortuje koszty według daty, następnie grupuje je według roku. Dla każdego roku wywoływana jest funkcja `costListItemsByMonths()` w celu wygenerowania listy elementów kosztów pogrupowanych według miesięcy. 

Funkcja `costListItemsByMonths()` przyjmuje rok i listę kosztów dla tego roku jako argument. Tworzy listę elementów kosztów według miesięcy, rozpoczynając od `CostYearItem`, który reprezentuje sam rok. Następnie koszty są grupowane według miesięcy, a funkcja `costListItemsByDays()` jest wywoływana w celu wygenerowania listy elementów kosztów pogrupowanych według dni w każdym miesiącu. 

Funkcja `costListItemsByDays()` przyjmuje miesiąc i listę kosztów dla tego miesiąca jako argument. Tworzy listę elementów kosztów według dni, rozpoczynając od `CostMonthItem`, który reprezentuje sam miesiąc. Następnie koszty są sortowane według dni, a każdy koszt jest mapowany na `CostGeneralItem`, który reprezentuje indywidualny koszt.

Funkcja `getTimeLineList()` zwraca listę elementów costListItem.

- `generalCosts` - to jest kolekcja kosztów, którą ta funkcja przetwarza.
- `sortedBy { it.date }` - ta funkcja sortuje koszty w `generalCosts` w porządku chronologicznym względem ich dat.
- `groupBy { it.date.year }` - ta funkcja grupuje koszty, zebrane w poprzednim kroku, po roku, w którym zostały poniesione. W ten sposób uzyskujemy mapę, w której kluczem jest rok, a wartością jest lista kosztów poniesionych w danym roku (`Map<Int, List<Cost>>`).
- `flatMap { (year, costsOfYear) -> costListItemsByYear(year, costsOfYear) }` - ta funkcja przetwarza każdą parę **klucz-wartość** z mapy utworzonej w poprzednim kroku. Funkcja `flatMap` *spłaszcza* wyniki do jednej listy. `costListItemsByYear(year, costsOfYear)` to funkcja, która tworzy listę elementów `costListItem`. Funkcja `flatMap` stosowana jest tutaj, aby połączyć wyniki zwracane przez każde wywołanie `costListItemsByYear` w jedną listę.

`costListItemsByMonths` jest funkcją, która tworzy listę elementów `CostListItem` dla danego roku i kosztów poniesionych w tym roku.

- `year: Int` - to jest rok, dla którego tworzymy elementy `CostListItem`.
- `costsOfYear: List<Cost>` - to jest lista kosztów poniesionych w danym roku.
- `listOf(...)` - ta funkcja tworzy listę elementów `CostListItem`. Na początku tej listy dodajemy element `CostYearItem`, który zawiera informacje o roku, dla którego tworzymy listę.
- `*` - operator ten rozpakowuje tablicę, a dokładniej mówiąc przekształca jej elementy na osobne argumenty.
- `costsOfYear.groupBy { it.date.month }` - ta funkcja grupuje koszty zebrane w poprzednim kroku po miesiącu, w którym zostały poniesione.
- `flatMap { (month, costsOfMonth) -> costListItemsByDays(month, costsOfMonth) }` - ta funkcja przetwarza każdą parę klucz-wartość z mapy utworzonej w poprzednim kroku. Funkcja `flatMap` *spłaszcza* wyniki do jednej listy. `costListItemsByDays(month, costsOfMonth)` to funkcja, która tworzy listę elementów `CostListItem` dla danego miesiąca i kosztów poniesionych w tym miesiącu.
- `toTypedArray()` - ta funkcja przekształca wynik w tablicę.

Wykorzystujemy tutaj funkcję `polishMonthsNames`, zdefiniowaną w pliku `DateMapperUtil`, która służy do wyświetlania nazw miesięcy po polsku.

In [ ]:
fun polishMonthsNames(month: Month): String {
    return when (month) {
        Month.JANUARY -> "STYCZEŃ"
        Month.FEBRUARY -> "LUTY"
        Month.MARCH -> "MARZEC"
        Month.APRIL -> "KWIECIEŃ"
        Month.MAY -> "MAJ"
        Month.JUNE -> "CZERWIEC"
        Month.JULY -> "LIPIEC"
        Month.AUGUST -> "SIERPIEŃ"
        Month.SEPTEMBER -> "WRZESIEŃ"
        Month.OCTOBER -> "PAŹDZIERNIK"
        Month.NOVEMBER -> "LISTOPAD"
        Month.DECEMBER -> "GRUDZIEŃ"
        else -> "unknown"
    }
}

W efekcie utworzymy tworzymy listę elementów dla `Column`, która zawiera obiekty o typie `CostListItem` (jednym z jego podtypów). Utworzymy trzy `Composable`'y, zawierające różne elementy `ui`. Chcemy wyświetlić posortowaną listę w formacie

```verbatim
<YEAR> - Composable1
<MONTH> - Composable2
<DATE> <COST> - Composable3
```

<img src="https://play-lh.googleusercontent.com/Lm6SO1jsUF-VjsrfkIV9x-qpPIkIOIvgWkBGmxyZuHWPSbkYMS0oLhgoBk9wLXq6Xw=w5120-h2880" width="150" />

## Nawigacja

Aplikacja składa się z trzech ekranów.

In [ ]:
sealed class Screens(
    val route: String,
    val title: String,
    val icon: ImageVector
) {
    object Overview : Screens("overview", "Overview", Icons.Default.Home)
    object TimeLine : Screens("timeline", "Time Line", Icons.Default.Timeline)
    object Calculators : Screens("calculators", "Calculators", Icons.Default.Calculate)
}

Będzeimy wykorzystywać `NavigationBar` z ikonami - same ikony przechowujemy jako obiekty typu `ImageVector`

Dodajmy nawigację, którą wywołamy w `MainActivity`

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun Navigation(){
    val navController = rememberNavController()
    Scaffold(
        bottomBar = { BottomMenu(navController = navController)},
        content = { NavGraph(paddingValues = it, navController = navController) }
    )
}

Funkcja `Scaffold` jest używana do tworzenia podstawowej struktury widoku aplikacji. Przyjmuje ona dwa parametry: `bottomBar`, który definiuje dolny pasek nawigacji, oraz `content`, który definiuje zawartość główną strony.

W `bottomBar` przekazujemy komponent ``BottomMenu`, który jest odpowiedzialny za renderowanie dolnego paska nawigacji. Przekazujemy również `navController` jako parametr, aby możliwe było przekierowywanie nawigacji w odpowiedzi na interakcje użytkownika.

W `content` przekazujemy komponent `NavGraph`, który definiuje strukturę nawigacji w aplikacji. Przekazujemy również `paddingValues` jako parametr, aby umożliwić dostosowanie wcięcia zawartości w `Scaffold`.

Dodajmy funkcję renderującą dolny pasek

In [ ]:
@Composable
fun BottomMenu(navController: NavHostController){
    val screens = listOf(
        Screens.Overview, Screens.TimeLine, Screens.Calculators
    )

    val navBackStackEntry by navController.currentBackStackEntryAsState()
    val currentDestination = navBackStackEntry?.destination

    NavigationBar{
        screens.forEach{screen ->
            NavigationBarItem(
                label = { Text(text = screen.title)},
                icon = { Icon(imageVector = screen.icon, contentDescription = "icon") },
                selected = currentDestination?.hierarchy?.any { it.route == screen.route } == true,
                onClick = {navController.navigate(screen.route)}
            )
        }
    }
}

Tworzymy listę `screens` zawierającą ekrany dostępne w aplikacji. Każdy ekran reprezentowany jest przez obiekt `Screen`. Używamy `navController` do pobrania aktualnego stanu nawigacji (`currentBackStackEntryAsState()`) oraz aktualnej docelowej lokalizacji (`currentDestination`).

W pętli `forEach` iterujemy przez listę i dla każdego ekranu tworzymy `NavigationBarItem`. Ustalamy etykietę (`label`), ikonę (`icon`), stan zaznaczenia (`selected`) w zależności od tego, czy aktualna lokalizacja nawigacji odpowiada danemu ekranowi, oraz przypisujemy funkcję obsługującą kliknięcie (`onClick`), która przekierowuje nawigację do odpowiedniego ekranu za pomocą `navController.navigate(screen.route)`.

Następnie tworzymy komponent odpowiedzialny za renderowanie głównej zawartości aplikacji, zależnej od aktualnej lokalizacji nawigacji

In [ ]:
@Composable
fun NavGraph(paddingValues: PaddingValues, navController: NavHostController){
    NavHost(
        navController = navController,
        startDestination = Screens.Overview.route
    ) {
        composable(route = Screens.Overview.route){ OverviewScreen(paddingValues) }
        composable(route = Screens.TimeLine.route){ TimeLineScreen(paddingValues) }
        composable(route = Screens.Calculators.route){ CalculatorsScreen() }
    }
}

Wewnątrz funkcji definiujemy główny kontener nawigacji (`NavHost`), który zarządza przemieszczaniem się między różnymi ekranami w zależności od aktualnej lokalizacji nawigacji. Przekazujemy `navController` jako parametr do `NavHost`, aby umożliwić nawigację między ekranami. Parametr `startDestination` wskazuje na początkowy ekran, który zostanie wyświetlony po uruchomieniu aplikacji. W tym przypadku jest to ekran `OverviewScreen`, który ma przypisaną ścieżkę `Screens.Overview.route`.

Za pomocą funkcji `composable` definiujemy poszczególne ekrany, które są dostępne w aplikacji. Dla każdego ekranu podajemy jego ścieżkę (`route`) i funkcję, która renderuje zawartość danego ekranu. Przekazujemy również `paddingValues` jako parametr, który może być wykorzystany przez ekrany do dostosowania wcięcia zawartości - zawartość ekranu `CalculatorsScreen` jest zawsze mniejsza niż rozmiar samego ekranu, więc pomijamy parametr`

Nawigację wywołujemy w `MainActivity`

In [ ]:
class MainActivity : ComponentActivity() {
    override fun onCreate(savedInstanceState: Bundle?) {
        super.onCreate(savedInstanceState)
        setContent {
            CarsyComposeTheme {
                // A surface container using the 'background' color from the theme
                Surface(
                    modifier = Modifier.fillMaxSize(),
                    color = MaterialTheme.colorScheme.background
                ) {
                    Navigation()
                }
            }
        }
    }
}

## OverviewScreen

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExY2MzYmNiN2NmOTE4NDVmZGU4ZTQzNjQ3ZjVkZDlhNzE3MTc0Y2FmOCZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/UT1CIBlbL1FftHf9Hk/giphy.gif" width="200" />

Ekran mamy podzielony na trzy sekcje
- Podumowanie kosztów - zawiera `LazyRow` którrego elementami są zestawienia kosztów dla każdego posiadanego samochodu
- Garaż - zawiera `LazyRow` którego elementami są detale wszystkich posiadanych samochodów
- Koszty całkowite - zawiera listę wszystkich poniesionych kosztów

Dodajmy trzy metody

In [ ]:
private fun totalValue(costType: CostType): String {
    return decimalFormat.format(
        DataProvider.cars
            .flatMap { it.costs }
            .filter { it.type == costType }
            .sumOf { it.amount }).toString()
}

In [ ]:
val decimalFormat = DecimalFormat("###,###.##")

Za pomocą metody `flatMap`, pobieramy wszystkie koszty ze wszystkich samochodów i łączymy je w jedną listę. Metoda `filter` filtruje koszty, aby wybrać tylko te, których typ odpowiada wartości `costType`. Na końcu, za pomocą metody `sumOf`, obliczamy sumę wartości dla wybranego typu kosztu. Wynik sumy jest sformatowany za pomocą `decimalFormat` i przekształcany na łańcuch znaków za pomocą metody `toString()`, który jest zwracany przez funkcję.

In [ ]:
private fun costValue(item: Car, costType: CostType) =
    decimalFormat.format(item.costs
        .filter { it.type == costType }
        .sumOf { it.amount }).toString()

Za pomocą metody `filter`, filtrujemy koszty, aby wybrać tylko te, których typ odpowiada wartości `costType`. Na końcu, za pomocą metody `sumOf`, obliczamy sumę wartości kosztów dla wybranego typu. Wynik sumy jest sformatowany za pomocą `decimalFormat` i przekształcany na łańcuch znaków za pomocą metody `toString()`, który jest zwracany przez funkcję.

In [ ]:
private fun getCarDetails(index: Int): List<Pair<String, String>> {
    var data: List<Pair<String, String>>
    DataProvider.cars[index].apply {
        data = listOf(
            "Marka:" to brand,
            "Model:" to model,
            "Rok:" to yearOfProduction.toString(),
            "Suma:" to decimalFormat.format(costs.sumOf { cost -> cost.amount })
                .toString() + " zł"
        )
    }
    return data
}

Funkcja `getCarDetails`, zwraca listę par wartości typu `Pair<String, String>`. Funkcja przyjmuje jeden argument `index` typu `Int`, który wskazuje indeks samochodu w liście `DataProvider.cars`. Zwraca listę par wartości reprezentujących szczegóły dotyczące samochodu.

Przejdźmy do utworzenia interfejsu użytkownika, całość umieścimy w `Column`

In [ ]:
Column(
    modifier = Modifier
        .padding(paddingValues)
        .verticalScroll(rememberScrollState())
) {)

Prrzekazujemy `paddingValues` aby zmieścić zawartość i nie przykrywać jej przez `bottomBar`, wykorzystujemy również `verticalScroll` aby umożliwić przewijanie wertykalne. `rememberScrollState()` pozwala przechować stan scrolla - czyli miejsce w którym znajdujemy się na liście.

Trzy sekcje będą znajdować się w kartach (`Card`) - elementy wewnętrzne posiadają własne karty. Pierwszym elementem każdej karty jest pasek tytułu, dodajmy dwie funkcje renderujące ten pasek dla zewnętrznych oraz wewnętrznych kart.

In [ ]:
@Composable
private fun CreateOuterCardTitle(title: String) {
    Box(
        modifier = Modifier
            .background(Color(128, 203, 196))
            .fillMaxWidth()
    ) {
        Text(
            text = title,
            textAlign = TextAlign.Center,
            modifier = Modifier.fillMaxWidth(),
            fontSize = 24.sp,
            fontWeight = FontWeight.Bold,
            style = TextStyle.Default.copy(Color.Black)
        )
    }
}

Funkcja `CreateOuterCardTitle` renderuje tytuł karty, który jest złożony z kontenera `Box` z tłem oraz komponentu `Text` z odpowiednimi stylami dla tekstu. Właściwość `modifier` komponentu `Box` jest odpowiedzialna za stylizację i układ. W tym przypadku używamy modyfikatora `background`, aby ustawić tło `Color(128, 203, 196)` dla kontenera. Modyfikator `fillMaxWidth()` rozciąga kontener na maksymalną szerokość dostępną. Wewnątrz znajduje się komponent `Text`, który renderuje tekst tytułu. Modyfikator `textAlign` ustawia wyśrodkowanie tekstu w kontenerze.  Określamy również rozmiar tekstu (`fontSize`), grubość tekstu (`fontWeight`), a także kolor tekstu (`style`), który jest skopiowany z domyślnego stylu, ale z innym kolorem (`Color.Black`).

Przejdźmy do funkcji renderującej wewnętrzny pasek tytułu dla kart.

In [ ]:
@Composable
private fun CreateInnerCardTitle(title: String, width: Int) {
    Box(
        modifier = Modifier
            .width(width.dp)
            .background(Color(166, 111, 131))
    ) {
        Text(
            text = title,
            textAlign = TextAlign.Center,
            modifier = Modifier.fillMaxWidth(),
            fontSize = 20.sp,
            fontWeight = FontWeight.Bold,
            style = TextStyle.Default.copy(Color.Black)
        )
    }
}

Funkcja przyjmuje dwa argumenty: `title` typu `String` oraz `width` typu `Int`. Szerokość `Box` jest ustawiony na podstawie wartości width (`width(width.dp)`), dodatkowo, modyfikator modifier dla komponentu Box ustawia `background` na `Color(166, 111, 131)`

Kolejnym elementem będzie komponent renderujący zawartość wewnętrznej karty.

In [ ]:
@Composable
private fun CreateInnerCardRow(firstText: String, secondText: String) {
    Row(modifier = Modifier.fillMaxWidth()) {
        Text(
            text = firstText,
            fontSize = 16.sp,
            modifier = Modifier.padding(start = 4.dp)
        )
        Spacer(modifier = Modifier.weight(1f))
        Text(
            text = secondText,
            fontSize = 16.sp,
            fontWeight = FontWeight.Bold,
            modifier = Modifier.padding(end = 4.dp)
        )
    }
}

Funkcja przyjmuje dwa argumenty typu `String`: `firstText` i `secondText`, które reprezentują tekst do wyświetlenia w pierwszej i drugiej kolumnie wiersza. Wewnątrz funkcji tworzymy wiersz (`Row`). Wewnątrz, definiujemy dwa teksty (`Text`). Pierwszy tekst (`firstText`) jest umieszczony w lewej kolumnie wiersza, a drugi tekst (`secondText`) jest umieszczony w prawej kolumnie wiersza. Dodajemy `Spacer` z właściwością `weight(1f)`, aby wypełnić dostępną przestrzeń między dwoma tekstami i rozciągnąć drugi tekst do końca wiersza.

Pierwsza karta będzie zawierała podsumowanie kosztów, rozpoczynamy od dodanie samej karty, oraz tytułu

In [ ]:
Card(
    modifier = Modifier
        .fillMaxWidth()
        .padding(12.dp)
) {
    CreateOuterCardTitle("Podsumowanie kosztów")

Następnie dodajmy `LazyRow`

In [ ]:
LazyRow(
    modifier = Modifier
        .fillMaxWidth()
        .padding(12.dp)
){
    items(DataProvider.cars.size){ index ->
        ...
    }
}

Każdy element `LazyRow` jest kartą zawierającą tytuł oraz listę elementów

In [ ]:
LazyRow(
    modifier = Modifier
        .fillMaxWidth()
        .padding(12.dp)
){
    items(DataProvider.cars.size){ index ->
        Card(
            modifier = Modifier
                .padding(4.dp)
                .fillMaxWidth(),
            elevation = CardDefaults.cardElevation(8.dp)
        ) {
            CreateInnerCardTitle(title = DataProvider.cars[index].name, width = 250)
            Column(
                modifier = Modifier
                    .width(250.dp)
                    .padding(bottom = 8.dp)
            ) {
                CostType.values().forEach {
                    CreateInnerCardRow(
                        it.costType,
                        costValue(DataProvider.cars[index], it) + " zł"
                    )
                }
            }
        }
    }
}

Dodajemy tytuł karty oraz tworzymy `Column`, w której umieszczamy wszystkie wartości dla danego kosztu. Dla każdego poniesionego typu kosztu dodajemy komponent `CreateInnerCardRow`

Podobnie tworzymy kolejną kartę reprezentującą garaż.

In [ ]:
Card(
    modifier = Modifier
        .fillMaxWidth()
        .padding(12.dp)
) {
    CreateOuterCardTitle(title = "Garaż")

    LazyRow(
        modifier = Modifier
            .fillMaxWidth()
            .padding(12.dp)
    ){
        items(DataProvider.cars.size){ index ->
            Card(
                modifier = Modifier
                    .padding(4.dp)
                    .fillMaxWidth(),
                elevation = CardDefaults.cardElevation(8.dp)
            ) {
                CreateInnerCardTitle(title = DataProvider.cars[index].name, width = 200)
                Column(
                    modifier = Modifier
                        .width(200.dp)
                        .padding(bottom = 8.dp)
                ) {
                    val data: List<Pair<String, String>> = getCarDetails(index)
                    data.forEach {
                        CreateInnerCardRow(
                            firstText = it.first,
                            secondText = it.second
                        )
                    }
                }
            }
        }
    }
}

In [ ]:
fun <T> List<T>.second(): T {
    return this[1]
}

fun <T> List<T>.third(): T {
    return this[2]
}

Ostatnią kartą są koszty całkowite.

In [ ]:
Card(
    modifier = Modifier
        .fillMaxWidth()
        .padding(12.dp)
) {
    CreateOuterCardTitle("Koszty całkowite")

    Card(
        modifier = Modifier
            .padding(4.dp)
            .fillMaxWidth(),
        elevation = CardDefaults.cardElevation(8.dp)
    ) {
        Column(
            modifier = Modifier
                .fillMaxWidth()
                .padding(bottom = 8.dp)
        ) {
            CostType.values().forEach {
                CreateInnerCardRow(it.costType, totalValue(it) + " zł")
            }
        }
    }
}

Mamy tylko jeden element, więc pomijamy tworzenie `LazyRow`

W efekcie nasz ekran wygląda nastepująco

<img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExY2MzYmNiN2NmOTE4NDVmZGU4ZTQzNjQ3ZjVkZDlhNzE3MTc0Y2FmOCZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/UT1CIBlbL1FftHf9Hk/giphy.gif" width="200" />

## TimeLineScreen

Ekran linii czasu jest nieco bardziej skomplikowany, mamy jedą listę elementów różnych typów - dla każdego typu chcemy wywołać inną funkcję `Composable`. Dodatkowo dodamy `DropDownMenu` aby umnożliwić wyświetlenie linii czasu dla wybranego pojazdu.

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExYWRlZGExYmI4OTZlNjY4MzZiM2NkZmE4ZWMzN2VlMGVjZjk4YWRhOSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/lx96i0NeTGeYV35NFl/giphy.gif" width="200" />

Rozpocznijmy od dodania `DropDownMenu` - samo pole składa się z `OutlinedTextField` oraz `DropDownMenu`. Wpierw potrzebujemy kilka zmiennych. 

Pierwszą są dane do przetwarzania.

In [ ]:
val cars = remember { DataProvider.cars }

Potrzebujemy również flagę przechowującą informację o stanie listy rozwijanej (rozwinięta/zwinięta)

In [ ]:
var expanded by remember { mutableStateOf(false) }

Konieczna będzie również zmienna przechowująca aktualnie wybraną opcję

In [ ]:
var selectedText by remember { mutableStateOf(cars.first().name) }

W celu wyświetlenia i zarządzania rozwijanym menu wykorzystamy również pole przechowujące aktualny rozmiar

In [ ]:
var textFieldSize by remember { mutableStateOf(Size.Zero) }

Potrzebujemy również dane do wyświetlenia na linii czasu

In [ ]:
val data = remember {
    mutableStateListOf(DataProvider.getTimeLineList(cars.first().costs)) // domyślnie wyświetlamy pierwszy z listy
}

Ostatnim elementem będzie ikona w polu tekstowym wskazująca czy lista jest zwinięta/rozwinięta - po kliknięciu następuje zmiana stanu

In [ ]:
val icon = if (expanded)
    Icons.Filled.ArrowUpward
else
    Icons.Filled.ArrowDownward

Zewnętrznym elementem jest kolumna bez modyfikatorów

In [ ]:
Column {}

Wewnątrz umieścimy listę rozwijaną oraz linię czasu.

Samą rozwijaną listę umieścimy również wc kolumnie - będzie zawierać, jak wcześniej wspomniałem, dwa elementy
- `OutlineTextField` - wyświetlanie aktualnie wybranego tekstu
- `DropDownMenu` - lista dostępnych opcji



In [ ]:
Column(Modifier.padding(20.dp)) {
    OutlinedTextField(
        ...
    )
    DropdownMenu(
        ...
    )
}

Rozpocznijmy od pola tekstowego

In [ ]:
OutlinedTextField(
    value = selectedText,
    readOnly = true,
    onValueChange = { selectedText = it },
    modifier = Modifier
        .fillMaxWidth()
        .onGloballyPositioned { coordinates ->
            textFieldSize = coordinates.size.toSize()
        },
    label = { Text("Kolekcja", fontSize = 18.sp) },
    trailingIcon = {
        Icon(icon, "contentDescription",
            Modifier.clickable { expanded = !expanded })
    },
    textStyle = TextStyle.Default.copy(fontSize = 24.sp, textAlign = TextAlign.Center)
)

- `value` - Określa wartość tekstową pola tekstowego. W tym przypadku `selectedText` jest zmienną przechowującą wartość pola tekstowego.

- `readOnly` Określa, czy pole tekstowe jest tylko do odczytu. Ustawione na `true` oznacza, że pole tekstowe nie może być edytowane.
- `onValueChange` Funkcja wywoływana przy zmianie wartości pola tekstowego. Aktualizuje wartość zmiennej `selectedText` na nową wartość wprowadzoną przez użytkownika - wybraną z listy.
- `modifier` Modyfikator określający wygląd i układ pola tekstowego. 
- `onGloballyPositioned` pozwala na przechwycenie informacji o rozmiarze pola tekstowego i przypisanie go do zmiennej textFieldSize. Funkcja jest wywoływana, gdy pole tekstowe zostaje umieszczone na ekranie i obliczone są jego współrzędne oraz rozmiar. Wartość `coordinates` przechowuje informacje o położeniu i rozmiarze pola tekstowego. W ten sposób możliwe jest przechowywanie informacji o rozmiarze pola tekstowego w zmiennej `textFieldSize` i wykorzystanie jej w innych częściach kodu - zostanie wykorzystana do ustalenia szerokości listy rozwijanej.
- `label` Etykieta pola tekstowego wyświetlana powyżej pola.
- `trailingIcon` Ikona wyświetlana na końcu pola tekstowego. Wykorzystuje komponent `Icon` z przekazaną ikoną i opisem treści. Dodatkowo, `Modifier.clickable` umożliwia obsługę kliknięcia na ikonę, co powoduje zmianę wartości zmiennej `expanded` i zwinięcie/rozwinięcie listy.
- `textStyle` Określa styl czcionki dla tekstu w polu tekstowym.

Następnie dodajmy `DroppDownMenu`

In [ ]:
DropdownMenu(
    expanded = expanded,
    onDismissRequest = { expanded = false },
    modifier = Modifier
        .width(with(LocalDensity.current) { textFieldSize.width.toDp() })
) {
    cars.map { it.name }.forEach { name ->
        DropdownMenuItem(
            ...
        )
    }
}

`onDismissRequest` jest funkcją obsługującą żądanie zamknięcia menu. W przypadku kliknięcia poza menu lub wywołania funkcji `onDismiss` menu zostanie zamknięte. Wartość `with(LocalDensity.current) { textFieldSize.width.toDp() }` konwertuje szerokość pola tekstowego przechowywaną w zmiennej `textFieldSize` na jednostki `Dp`, co umożliwia elastyczne dostosowanie szerokości menu do szerokości pola tekstowego. Dodajmy element do menu.

In [ ]:
DropdownMenuItem(
    text = {
        Text(
            text = name,
            fontSize = 24.sp,
            textAlign = TextAlign.Center,
            modifier = Modifier.fillMaxWidth()
        )
           },
    onClick = {
        selectedText = name
        expanded = false
        data.clear()
        data.add(DataProvider.getTimeLineList(cars.find { it.name == selectedText }!!.costs))
    },
    modifier = Modifier.fillMaxWidth()
)

Po kliknięciu przypisujemy nazwę do `selectedText`, zmieniamy flagę `expanded`, czyścimy dane oraz dodajemy nową linię czasu, odpowiadającą wybranemu elementowi.

Następnie dodamy `LazyColumn`, wyświetlający nasze dane.

In [ ]:
LazyColumn(
    modifier = Modifier
        .padding(paddingValues)
        .fillMaxSize(),
    verticalArrangement = Arrangement.Center,
    horizontalAlignment = Alignment.CenterHorizontally
) {
    items(data.first().size) { index ->
        when (data.first()[index]) {
            is CostListItem.CostYearItem -> YearItem(item = data.first()[index] as CostListItem.CostYearItem)
            is CostListItem.CostMonthItem -> MonthItem(item = data.first()[index] as CostListItem.CostMonthItem)
            is CostListItem.CostGeneralItem -> GeneralItem(item = data.first()[index] as CostListItem.CostGeneralItem)
        }
    }
}

W zależności od typu pola (`CostYearItem`, `CostMonthItem`, `CostGeneralItem`), chcemy wyświetlić element z innym layoutem - gdyby wykorzystane było tylko grupowanie po jednym elemencie, można wykorzystać `Sticky headers` z biblioteki `Compose`, ponieważ chcemy grupowac w pierwszej kolejności po roku, następnie po miesiącu, musimy to zaimplementować od podstaw.

Dodajmy te trzy `Composable`, zacznijmy od wyświetlania roku - tutaj chcemy tylko wycentrowane pole tekstowe

In [ ]:
@Composable
private fun YearItem(item: CostListItem.CostYearItem) {
    Text(
        text = item.year,
        fontSize = 36.sp,
        fontWeight = FontWeight.Bold,
        color = Color(128, 203, 196),
        modifier = Modifier
            .height(66.dp)
            .padding(top = 12.dp)
    )
}

Komponent reprezentujący miesiąc zawiera jego nazwę oraz dwa elementy graficzne - linię oraz ikonę (kółko).

In [ ]:
@Composable
private fun MonthItem(item: CostListItem.CostMonthItem) {
    Row(
        modifier = Modifier
            .fillMaxWidth()
            .height(40.dp)
    ) {
        Box(modifier = Modifier
            .width(60.dp)
            .fillMaxHeight()
            .drawWithContent {
                drawContent()
                drawLine(
                    color = Color(128, 203, 196),
                    start = Offset(size.width / 2, 0f),
                    end = Offset(size.width / 2, size.height),
                    strokeWidth = 4.dp.toPx()
                )
            }
        ) {
            Image(
                imageVector = Icons.Default.Circle,
                contentDescription = "",
                modifier = Modifier
                    .fillMaxWidth()
                    .padding(top = 6.dp),
                colorFilter = ColorFilter.tint(Color(128, 203, 196))
            )

        }
        Text(
            text = item.month,
            color = Color(166, 111, 131),
            fontSize = 24.sp,
            fontWeight = FontWeight.Bold,
            textAlign = TextAlign.Center,
            modifier = Modifier.fillMaxWidth(.8f)
        )
    }
}

W części `Box` jest rysowana jest pozioma linia wyśrodkowana wewnątrz prostokątnego obszaru o szerokości `60.dp` i pełnej wysokości. Linia ma kolor `(128, 203, 196)` i grubość `4.dp`. Ponad obszarem znajduje się ikona (`Icons.Default.Circle`), który ma ustawiony kolor na `(128, 203, 196)` - `Box` pozwala na umieszczanie elementów jeden na drugim. W części `Text` znajduje się tekst reprezentujący nazwę miesiąca.

Metoda `drawWithContent` jest wywoływana na modifiekatorze `Box` i pozwala na rysowanie własnych treści wewnątrz komponentu. Wewnątrz metody najpierw wywoływana jest funkcja `drawContent()`, która rysuje oryginalną zawartość komponentu. Następnie wywoływana jest funkcja `drawLine`, która rysuje poziomą linię środkową.

- `start` Punkt początkowy linii, który jest ustawiony na połowę szerokości prostokątnego obszaru (`size.width / 2`) i wysokość `0f`.
- `end` Punkt końcowy linii, który jest ustawiony na połowę szerokości prostokątnego obszaru (`size.width / 2`) i wysokość `size.height`.
- `strokeWidth` Grubość linii, która jest ustawiona na wartość `4.dp.toPx()`, czyli przeliczenie `4.dp` na piksele.

Ostatnim elementem jest `GeneralItem`, tuitaj w komponencie `Box` chcemy umieścić linię jako pierwszy element, kółko jako drugi, oraz ikonę jako trzeci.

In [ ]:
@Composable
private fun GeneralItem(item: CostListItem.CostGeneralItem) {
    Row(
        modifier = Modifier.height(68.dp),
        verticalAlignment = Alignment.CenterVertically
    ) {
        Box(
            modifier = Modifier
                .width(60.dp)
                .fillMaxHeight(),
            contentAlignment = Alignment.Center
        ) {
            Canvas(modifier = Modifier.fillMaxSize()) {
                drawLine(
                    color = Color(128, 203, 196),
                    start = Offset(size.width / 2, 0f),
                    end = Offset(size.width / 2, size.height),
                    strokeWidth = 4.dp.toPx()
                )
            }
            Image(
                imageVector = Icons.Default.Circle,
                contentDescription = "",
                modifier = Modifier
                    .size(48.dp)
                    .fillMaxWidth()
                    .padding(top = 6.dp),
                colorFilter = ColorFilter.tint(Color(128, 203, 196))
            )

            Image(
                imageVector = item.cost.type.icon,
                contentDescription = "",
                modifier = Modifier
                    .size(24.dp)
                    .padding(top = 4.dp)
            )

        }
        Column(Modifier) {
            Text(
                text = item.cost.type.costType,
                fontSize = 20.sp,
                color = Color(128, 203, 196)
            )
            Text(
                text = item.cost.date.format(dateFormatter),
                fontSize = 12.sp,
                color = Color(128, 203, 196)
            )
        }
        Spacer(Modifier.weight(1f))
        Text(
            text = item.cost.amount.toString().format(decimalFormat) + " zł",
            modifier = Modifier
                .align(Alignment.CenterVertically)
                .padding(end = 12.dp),
            fontSize = 24.sp,
            fontWeight = FontWeight.Bold,
            color = Color(128, 203, 196)
        )
    }
}

In [ ]:
val dateFormatter: DateTimeFormatter = DateTimeFormatter.ofPattern("dd MMM yyyy", Locale("pl"))

Pierwszy element to `Box`, który zawiera linię środkową oraz ikony okręgu i ikony typu kosztu. Linia środkowa jest rysowana przy użyciu komponentu `Canvas` i funkcji `drawLine`. Ikony okręgu i typu kosztu są renderowane przy użyciu komponentu `Image`. Ustawione są odpowiednie kolory i rozmiary dla tych elementów. Drugi element to `Column`, który zawiera dwa teksty: typ kosztu i data. Teksty są renderowane przy użyciu komponentu `Text`. Trzeci element to `Spacer` z ustawioną wagą `1f`, co powoduje, że zajmuje dostępne dostępne miejsce i przesuwa następne elementy na prawą stronę. Ostatnim elementem jest `Text`, który wyświetla wartość kosztu. Ustawione są odpowiednie style, rozmiar, waga czcionki i kolor.

W efekcie nasz ekran wygląda nastepująco

<img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExYWRlZGExYmI4OTZlNjY4MzZiM2NkZmE4ZWMzN2VlMGVjZjk4YWRhOSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/lx96i0NeTGeYV35NFl/giphy.gif" width="200" />

## CalculatorsScreen

Ostatni ekran będzie zawierał kilka kalkulatorów wybieranych przez `DropDownMenu`. 

<img src="https://media3.giphy.com/media/v1.Y2lkPTc5MGI3NjExZDFmZTkxZjIxNzQ0NTJmNDFlM2NhN2NiNTlmYzU2N2ZkZTVhODM3MyZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/WyCanQMjlfmYguKc3l/giphy.gif" width="200" />

Będziemy się posługiwali edytowalnymi polami, obliczenia zostaną wykonane po naciśnięciu przycisku. Wpierw dodajmy funkcję sprawdzającą, czy wszystkie pola zostały wypełnione.

In [ ]:
fun areFieldsEmptyOrNull(fieldTop: String?, fieldMiddle: String?, fieldBottom: String?): Boolean {
    return fieldTop.isNullOrEmpty() || fieldMiddle.isNullOrEmpty() || fieldBottom.isNullOrEmpty()
}

Dodajmy również metody wykonujące obliczenia.

In [ ]:
private fun fuelCalculator(values: Triple<String, String, String>): Pair<String, String> {

    val distance = values.first.toDouble()
    val cost = values.second.toDouble()
    val fuelUsage = values.third.toDouble()

    val fuelCost = (distance * fuelUsage) / 100
    val fuelAmount = fuelCost * cost

    val stringMain = decimalFormat.format(fuelCost).toString()
    val stringBottom = decimalFormat.format(fuelAmount).toString()

    return Pair("$stringMain l", "$stringBottom zł")
}

private fun distanceCalculator(values: Triple<String, String, String>): Pair<String, String> {

    val fuel = values.first.toDouble()
    val price = values.second.toDouble()
    val fuelUsage = values.third.toDouble()

    val distance = fuel * price
    val totalCost = (100 * fuel) / fuelUsage

    val stringBottom = decimalFormat.format(distance).toString()
    val stringMain = decimalFormat.format(totalCost).toString()

    return Pair("$stringMain km", "$stringBottom zł")
}

private fun travelCalculator(values: Triple<String, String, String>): Pair<String, String> {

    val distance = values.first.toDouble()
    val price = values.second.toDouble()
    val fuelUsage = values.third.toDouble()

    val totalFuel = (fuelUsage / 100) * distance
    val totalPrice = totalFuel * price

    val stringBottom = decimalFormat.format(totalFuel).toString()
    val stringMain = decimalFormat.format(totalPrice).toString()

    return Pair("$stringMain zł", "$stringBottom l")
}

Podobnie jak poprzednio, potrzebujemy kilka pól do obsługi rozwijanej listy.

In [ ]:
val calculatorItems = listOf(
    "Koszt podróży" to listOf("Odległość [km]", "Cena za litr [PLN]", "Spalanie [l/100km]"),
    "Wymagane paliwo" to listOf("Odległość [km]", "Cena za litr [PLN]", "Spalanie [l/100km]"),
    "Odległość" to listOf("Paliwo [l]", "Cena za listr [PLN]", "Spalanie [l/100km]")
)

var expanded by remember { mutableStateOf(false) }

var selectedText by remember { mutableStateOf(calculatorItems.first().first) }

var textFieldSize by remember { mutableStateOf(Size.Zero) }

val icon = if (expanded)
    Icons.Filled.ArrowUpward
else
    Icons.Filled.ArrowDownward

Kolejne pole przechowuje informacje o etykietach w polach tekstowych - inicjujemy pierwszym elementem.

In [ ]:
val labelData = remember { mutableStateListOf(calculatorItems.first().second) }

Ostatnim elementem są pola przechowujące informajce oraz pokazujące wyniki obliczeń

In [ ]:
var fieldTop by remember { mutableStateOf("")}
var fieldMiddle by remember { mutableStateOf("")}
var fieldBottom by remember { mutableStateOf("")}

var mainResult by remember { mutableStateOf("")}
var subResult by remember { mutableStateOf("")}

Przejdźmy do layoutu - całość umieszczamy w `Column`. `DropDownMenu` wygląda jak w poprzednim ekranie.

In [ ]:
Column(Modifier.padding(20.dp)) {
    OutlinedTextField(
        value = selectedText,
        readOnly = true,
        onValueChange = { selectedText = it },
        modifier = Modifier
            .fillMaxWidth()
            .onGloballyPositioned { coordinates ->
                textFieldSize = coordinates.size.toSize()
            },
        label = { Text("Kalkulatory", fontSize = 18.sp) },
        trailingIcon = {
            Icon(icon, "contentDescription",
                Modifier.clickable { expanded = !expanded })
        },
        textStyle = TextStyle.Default.copy(fontSize = 24.sp, textAlign = TextAlign.Center)
    )
    CreateDropDownMenu(
        expanded = expanded,
        textFieldSize =  textFieldSize,
        calculatorItems = calculatorItems,
        onExpandedChange = {expanded = false},
        onClick = { selected ->
            selectedText = selected
            expanded = false
            labelData.clear()
            labelData.add(calculatorItems.find { it.first == selectedText }!!.second)
        }
    )
}

In [ ]:
@Composable
private fun CreateDropDownMenu(
    expanded: Boolean,
    textFieldSize: Size,
    calculatorItems: List<Pair<String, List<String>>>,
    onExpandedChange: () -> Unit,
    onClick: (String) -> Unit
) {
    DropdownMenu(
        expanded = expanded,
        onDismissRequest = { onExpandedChange() },
        modifier = Modifier
            .width(with(LocalDensity.current) { textFieldSize.width.toDp() })
    ) {
        calculatorItems.map { it.first }.forEach { name ->
            DropdownMenuItem(
                text = {
                    Text(
                        text = name,
                        fontSize = 24.sp,
                        textAlign = TextAlign.Center,
                        modifier = Modifier.fillMaxWidth()
                    )
                },
                onClick = {onClick(name)}
                ,modifier = Modifier.fillMaxWidth()
            )
        }
    }
}

Wszystkie pola tekstowe umieszczamy na karcie.

In [ ]:
Card(
    modifier = Modifier
        .padding(12.dp)
        .fillMaxWidth(),
    shape = RoundedCornerShape(8.dp)
) {

Wewnątrez karty dodajemy trzy pola tekstowe - dane wejściowe dla obliczeń, oraz pola pokazujące wyniki.

In [ ]:
Column(
    modifier = Modifier
        .fillMaxWidth()
        .padding(8.dp),
    verticalArrangement = Arrangement.Center,
    horizontalAlignment = Alignment.CenterHorizontally
) {
    CreateOutlinedTextField(
        value = fieldTop,
        onValueChange = { fieldTop = it },
        labelText = labelData.first().first()
    )
    CreateOutlinedTextField(
        value = fieldMiddle,
        onValueChange = { fieldMiddle = it },
        labelText = labelData.first().second()
    )
    CreateOutlinedTextField(
        value = fieldBottom,
        onValueChange = { fieldBottom = it },
        labelText = labelData.first().third()
    )
    CreateBottomLabels(
        selectedText = selectedText,
        mainResult = mainResult,
        subResult = subResult
    )
}

In [ ]:
@OptIn(ExperimentalMaterial3Api::class)
@Composable
fun CreateOutlinedTextField(
    value: String,
    onValueChange: (String) -> Unit,
    labelText: String,
    modifier: Modifier = Modifier,
) {
    OutlinedTextField(
        value = value,
        singleLine = true,
        onValueChange = onValueChange,
        keyboardOptions = KeyboardOptions(keyboardType = KeyboardType.Number),
        modifier = modifier.fillMaxWidth(.8f),
        label = { Text(text = labelText, fontSize = 16.sp) },
        textStyle = TextStyle.Default.copy(fontSize = 24.sp, textAlign = TextAlign.Center)
    )
}

In [ ]:
@Composable
private fun CreateBottomLabels(
    selectedText: String,
    mainResult: String,
    subResult: String
) {
    Row(
        modifier = Modifier
            .fillMaxWidth()
            .padding(top = 12.dp)
    ) {
        Text(
            text = selectedText,
            modifier = Modifier.padding(start = 8.dp),
            fontSize = 28.sp
        )
        Spacer(modifier = Modifier.weight(1f))
        Column {
            Text(
                text = mainResult,
                fontSize = 22.sp,
                textAlign = TextAlign.End,
                modifier = Modifier.fillMaxWidth()
            )
            Text(
                text = subResult,
                fontSize = 16.sp,
                textAlign = TextAlign.End,
                modifier = Modifier.fillMaxWidth()
            )
        }
    }
}

Wartości pól przechowujemy we wcześniej zdefiniowanych polach `fieldTop`, `fieldModdle`, `fieldBottom`. Etykiety przechowujemy w zmiennej `labelData`

Poza kartą dodajemy przycisk, za pomocą którego wykonujemy obliczenia.

In [ ]:
CreateCalculateButton(
    selectedText =  selectedText,
    calculatorItems =  calculatorItems,
    fieldTop =  fieldTop,
    fieldMiddle =  fieldMiddle,
    fieldBottom = fieldBottom,
    onMainResultChange = {mainResult = it},
    onSubResultChange = {subResult = it}
)

In [ ]:
@Composable
private fun CreateCalculateButton(
    selectedText: String,
    calculatorItems: List<Pair<String, List<String>>>,
    fieldTop: String,
    fieldMiddle: String,
    fieldBottom: String,
    onMainResultChange: (String) -> Unit,
    onSubResultChange: (String) -> Unit
) {
    Row(modifier = Modifier.fillMaxWidth()) {
        Spacer(modifier = Modifier.weight(1f))
        Button(
            onClick = {
                onClickCalculateButton(
                    selectedText = selectedText,
                    calculatorItems = calculatorItems,
                    fieldTop = fieldTop,
                    fieldMiddle = fieldMiddle,
                    fieldBottom = fieldBottom,
                    onMainResultChange = onMainResultChange,
                    onSubResultChange = onSubResultChange
                )
            },
            modifier = Modifier.padding(top = 12.dp, end = 12.dp),
            shape = RoundedCornerShape(8.dp),
            colors = ButtonDefaults.buttonColors(containerColor = Color.Red)
        ) {
            Text(
                text = "OBLICZ",
                color = Color.White,
                fontSize = 16.sp,
                fontWeight = FontWeight.Bold
            )
        }
    }
}

In [ ]:
private fun onClickCalculateButton(
    selectedText: String,
    calculatorItems: List<Pair<String, List<String>>>,
    fieldTop: String,
    fieldMiddle: String,
    fieldBottom: String,
    onMainResultChange: (String) -> Unit,
    onSubResultChange: (String) -> Unit
) {
    val isEmptyOrNull = areFieldsEmptyOrNull(fieldTop, fieldMiddle, fieldBottom)
    if (isEmptyOrNull) {
        onMainResultChange("")
        onSubResultChange("")
    }
    else {
        val result = when (selectedText) {
            calculatorItems.first().first -> travelCalculator(
                Triple(fieldTop, fieldMiddle, fieldBottom)
            )

            calculatorItems.second().first -> fuelCalculator(
                Triple(fieldTop, fieldMiddle, fieldBottom)
            )

            else -> distanceCalculator(Triple(fieldTop, fieldMiddle, fieldBottom))
        }

        onMainResultChange(result.first)
        onSubResultChange(result.second)
    }
}

Funkcja `onClickCalculateButton` jest odpowiedzialna za obsługę kliknięcia przycisku *OBLICZ*. Przyjmuje kilka parametrów, takich jak `selectedText` (tekst wybrany z rozwijanej listy), `calculatorItems` (lista par zawierających nazwę kalkulatora i listę pól), `fieldTop`, `fieldMiddle` i `fieldBottom` (wartości wprowadzone przez użytkownika), oraz funkcje `onMainResultChange` i `onSubResultChange`, które aktualizują wynik główny i wynik pomocniczy.

Jeśli pola są puste lub mają wartość `null`, funkcja wywołuje funkcje `onMainResultChange` i `onSubResultChange` z pustymi wartościami, aby wyczyścić wyniki.

W przeciwnym razie, funkcja wybiera odpowiedni kalkulator na podstawie `selectedText` i wywołuje odpowiednią funkcję (`travelCalculator`, `fuelCalculator`, `distanceCalculator`) z wartościami pól jako argumenty. Wynik jest zwracany jako para wartości i przekazywany do funkcji `onMainResultChange` i `onSubResultChange`, które aktualizują wyniki w komponencie nadrzędnym.

Mopżemy przetestować aplikację.

<table><tr><td><img src="https://media1.giphy.com/media/v1.Y2lkPTc5MGI3NjExY2MzYmNiN2NmOTE4NDVmZGU4ZTQzNjQ3ZjVkZDlhNzE3MTc0Y2FmOCZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/UT1CIBlbL1FftHf9Hk/giphy.gif" width="200" /></td><td><img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExYWRlZGExYmI4OTZlNjY4MzZiM2NkZmE4ZWMzN2VlMGVjZjk4YWRhOSZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/lx96i0NeTGeYV35NFl/giphy.gif" width="200" /></td><td><img src="https://media3.giphy.com/media/v1.Y2lkPTc5MGI3NjExZDFmZTkxZjIxNzQ0NTJmNDFlM2NhN2NiNTlmYzU2N2ZkZTVhODM3MyZlcD12MV9pbnRlcm5hbF9naWZzX2dpZklkJmN0PWc/WyCanQMjlfmYguKc3l/giphy.gif" width="200" /></td></tr></table>